In [1]:
import os, glob, platform, datetime, random
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.autograd import Variable
from torch import functional as F
# import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

import cv2
from PIL import Image
from tensorboardX import SummaryWriter

import numpy as np
from numpy.linalg import inv as denseinv
from scipy import sparse
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import inv as spinv
import scipy.misc

from myimagefolder import MyImageFolder
from mymodel_basic import PreTrainedModel, GradientNet
from myargs import Args

In [2]:
# https://discuss.pytorch.org/t/on-a-cpu-device-how-to-load-checkpoint-saved-on-gpu-device/349
# snapshot = torch.load('snapshot_store/concat/snapshot-239.pth.tar')
snapshot = torch.load('./snapshot_store/1 basic/snapshot-179.pth.tar')


In [3]:
# torch.cuda.set_device(0)
state_dict = snapshot['state_dict']
args = snapshot['args']
densenet = models.__dict__[args.arch](pretrained=True).cuda(3)
pretrained = PreTrainedModel(densenet).cuda(3)
net = GradientNet(pretrained).cuda(3)
net.load_state_dict(state_dict)

In [4]:
pretrained.train()
net.train()

GradientNet (
  (pretrained_model): PreTrainedModel (
    (net_16M): Sequential (
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu0): ReLU (inplace)
    )
    (net_8M): Sequential (
      (pool0): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    )
    (net_4M): Sequential (
      (denseblock1): _DenseBlock (
        (denselayer1): _DenseLayer (
          (norm.1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
          (relu.1): ReLU (inplace)
          (conv.1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm.2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
          (relu.2): ReLU (inplace)
          (conv.2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer (
          (norm.1): BatchNorm2d(96, eps=1e-05, momentu

In [5]:
def loadimg(path):
    im = Image.open(path).convert('RGB')
#     im = im.resize((512, 256))
    print(im.size)
    im = transforms.ToTensor()(im)
    x = torch.zeros(1,3,416,416)
    x[0,:,:,:] = im[:,0:416,500:500+416]
    x = Variable(x)
    return x

im = loadimg('/home/lwp/workspace/sintel2/clean/alley_1/frame_0010.png')
# gt = cv2.imwrite('/home/cad/lwp/workspace/dataset/sintel2/albedo/alley_1/frame_0010.png')

# ft_pretrained = pretrained(im.cuda(3))
predict = net(im.cuda(3))


(1024, 436)


In [6]:
merged = predict[5]
merged = merged[0]
merged = merged.cpu().data.numpy()
print (merged.shape)
merged = merged.transpose(1,2,0)
print (merged.shape)
cv2.imwrite('out_merge.png', merged[:,:,::-1]*255)


(3, 416, 416)
(416, 416, 3)


True

In [ ]:
merged.shape

In [ ]:
%clear -a -f